In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import misc as ms
from fitter import model_average as md
from fitter import fitter as fit

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

C:\Users\anono\Anaconda2\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
reload(fit)
reload(dl)
reload(ms)
reload(md)

<module 'fitter.model_average' from '../fitter\model_average.pyc'>

In [3]:
data_loader = dl.data_loader()
fit_results = data_loader.get_fit_info('fit_results.csv')
other_results = data_loader.get_fit_info('other_results.csv')

In [4]:
model_average = md.model_average(fit_results)

In [7]:
model_average.average(split_unc=True, param='delta_su2')

[-0.0038325865269626226, 0.0006842062439685381, 0.00010190789108712172]

### Make plots

In [71]:
model_average.plot_fits('a');

In [16]:
model_average.plot_histogram('FK/Fpi');

In [73]:
model_average.plot_comparison(other_results=other_results, param='FK/Fpi_pm');